In [ ]:
import pandas as pd

In [ ]:
column_names = data.columns
print(column_names)

In [ ]:
agg_fn = {
    "type": lambda x: "; ".join(sorted(x.unique())),
    "required": lambda x: "; ".join(x.astype(str)),
    
}
print("before agg")
print(df)


df = df.groupby("", as_index=False).agg(agg_fn)

print("after agg")
print(df)

In [ ]:
table_a['name'] = table_a['name'].map(
                              dept_mapping.set_index('').idxmax(1))

In [ ]:
indexed_df = df.copy().set_index('required')
test = indexed_df.filter(lambda x: x['required'] == True)